In [1]:
import bitdotio
import os
import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
load_dotenv()
os.getenv("BITIO_USER")

BITIO_REPO = "cop26_methane"

# Methane Emissions by Sector
Source: https://cfpub.epa.gov/ghgdata/inventoryexplorer/#iallsectors/allsectors/methane/inventsect/all
"Where does methane come from"

In [9]:
df = pd.read_csv("../data/epa_methane_by_sector.csv")

# convert wide to long
df = pd.melt(df, id_vars=["Emissions of Methane, MMT CO2 eq."], var_name="Year", value_name="mmt_co2_eq")
df.rename(columns={"Year": "year", "Emissions of Methane, MMT CO2 eq.": "sector"}, inplace=True)

engine = create_engine(os.getenv("PG_STRING"))

with engine.connect() as conn:
    df.to_sql(
    "epa_methane_sources",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")

# Emissions by Gas

In [15]:
df = pd.read_csv("../data/epa_emissions_by_gas.csv")
df

,"Emissions by Gas, MMT CO2 eq.",1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Carbon dioxide,5113.454675,5057.930940,5167.480847,5267.343060,5358.824048,5421.502465,5610.581589,5686.464995,5731.049595,...,5675.786435,5540.173478,5338.698344,5474.256770,5522.807202,5371.770557,5248.023931,5207.751278,5375.490614,5255.816207
1,Methane,776.872695,781.779433,780.519139,770.710149,777.032032,767.697234,760.808472,746.757333,731.667549,...,692.149395,666.172529,658.159692,654.406243,651.027257,651.534655,642.389534,648.382337,655.936229,659.712703
2,Nitrous oxide,452.657524,443.229443,443.066108,471.293158,456.990572,468.807734,480.769690,466.844894,467.696827,...,454.950845,445.575097,416.764456,463.858788,473.988776,468.244115,450.792263,446.271465,459.211750,457.141519
3,Fluorinated gases,99.665733,90.681319,95.277742,95.003522,98.976178,117.852811,129.005625,136.440818,152.855296,...,168.219667,175.481316,172.285274,172.147308,177.133474,179.562738,179.132111,180.886250,180.810808,185.674751
4,Total,6442.650626,6373.621135,6486.343836,6604.349889,6691.822829,6775.860245,6981.165376,7036.508040,7083.269267,...,6991.106343,6827.402421,6585.907766,6764.669110,6824.956709,6671.112065,6520.337839,6483.291331,6671.449401,6558.345179


In [18]:
df = pd.read_csv("../data/epa_emissions_by_gas.csv")
# convert wide to long
df = pd.melt(df, id_vars=["Emissions by Gas, MMT CO2 eq."], var_name="Year", value_name="mmt_co2_eq")
df.rename(columns={"Year": "year", "Emissions of Methane, MMT CO2 eq.": "sector"}, inplace=True)

engine = create_engine(os.getenv("PG_STRING"))

with engine.connect() as conn:
    df.to_sql(
    "epa_emissions_by_gas",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")

# GWP of Methane
"Why do we care"
Source: https://www.ipcc.ch/report/ar5/wg1/

In [11]:
# get GWPs from HFC repo
engine = create_engine(os.getenv("PG_STRING"))
# SQL for querying an entire table
sql = f'''
    SELECT *
    FROM "bitdotio/hfc"."global_warming_potentials";
'''
# Return SQL query as a pandas dataframe
with engine.connect() as conn:
    # Set 1 minute statement timeout (units are milliseconds)
    conn.execute("SET statement_timeout = 60000;")
    df = pd.read_sql(sql, conn)

    df.to_sql(
    "ipcc_global_warming_potentials",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")

# International Comparison

In [14]:
unfccc_inventory = pd.read_csv("../data/processed_comparison.csv")
engine = create_engine(os.getenv("PG_STRING"))

with engine.connect() as conn:
    unfccc_inventory.to_sql(
    "unfccc_inventory",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")